*Data Engineer Project with Python*
Main objective: To create a complete database and tables to read CSV and insert the data into Postgres in Python.

In [1]:
#Installation of necessary packages.
#%pip install pandas
#%pip install psycopg2

In [2]:
import psycopg2
import pandas as pd

In [3]:
def create_database():
    # Connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=1234")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    # Create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE tickers")    
    cur.execute("CREATE DATABASE tickers")
    
    # Close connection to default database
    conn.close()

    # Connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=tickers user=postgres password=1234")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    return cur, conn

In [4]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [6]:
#Read the CSV extracted from Fundamentus.com.br
FundamentusStocks = pd.read_csv("FundamentusStocks20221126.csv",delimiter=";")

In [7]:
FundamentusStocks.head()

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,AESB3,9.81,-1443.07,1.41,2.086,0.009,0.389,1.98,11.53,-1.04,...,10.25,0.1810,0.0537,3.63,0.0448,-0.0010,20192400.0,4175170000,1.89,13.0240
1,ONCO3,7.34,-382.42,1.82,1.005,0.000,0.503,5.94,10.58,-1.39,...,11.79,0.0949,0.0109,1.42,0.0557,-0.0047,7053540.0,2014190000,1.48,0.0000
2,RAIL3,18.91,-316.18,2.33,3.839,0.001,0.770,7.63,15.86,-1.75,...,10.84,0.2421,-0.0124,1.83,0.0577,-0.0074,287566000.0,15061000000,1.11,0.0770
3,DMMO3,1.82,-252.76,-1.68,2.789,0.000,3.270,11.63,6.23,-1.34,...,5.29,0.4481,-0.0110,2.25,0.7039,0.0066,19918900.0,-553275000,0.00,-0.1737
4,ELMD3,10.16,-173.46,1.92,2.033,0.000,0.713,-7.09,16.34,-1.89,...,10.29,0.1244,0.0024,0.64,0.0504,-0.0111,3007740.0,739445000,0.91,1.0110


In [8]:
FundamentusStocks.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [9]:
#Create a cleaner dataframe with most important columns
FundamentusStocks_clean = FundamentusStocks[['Papel', 'P/L', 'P/VP', 'Div.Yield', 'EV/EBITDA']]

In [10]:
FundamentusStocks_clean.head()

,Papel,P/L,P/VP,Div.Yield,EV/EBITDA
0,AESB3,-1443.07,1.41,0.009,10.25
1,ONCO3,-382.42,1.82,0.000,11.79
2,RAIL3,-316.18,2.33,0.001,10.84
3,DMMO3,-252.76,-1.68,0.000,5.29
4,ELMD3,-173.46,1.92,0.000,10.29


In [11]:
FundamentusReits = pd.read_csv("FundamentusReits20221126.csv",delimiter=";")

In [12]:
FundamentusReits.head()

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média
0,ABCP11,Shoppings,72.01,0.0980,0.0935,0.78,878798000,78309,1,10109.50,1126.44,0.1114,0.0395
1,AFHI11,Títulos e Val. Mob.,94.34,0.0858,0.1559,0.99,284855000,844259,0,0.00,0.00,0.0000,0.0000
2,AGRX11,Híbrido,10.35,0.0476,0.0325,1.00,48910500,15035,0,0.00,0.00,0.0000,0.0000
3,AIEC11,Lajes Corporativas,71.65,0.1165,0.1240,0.70,345710000,564181,2,14750.80,2064.43,0.1400,0.0000
4,ALMI11,Lajes Corporativas,920.00,0.0148,0.0000,0.41,102283000,164370,1,2273.99,166.13,0.0731,0.5890


In [13]:
FundamentusReits.columns

Index(['Papel', 'Segmento', 'Cotação', 'FFO Yield', 'Dividend Yield', 'P/VP',
       'Valor de Mercado', 'Liquidez', 'Qtd de imóveis', 'Preço do m2',
       'Aluguel por m2', 'Cap Rate', 'Vacância Média'],
      dtype='object')

In [14]:
#Create a cleaner dataframe dropping columns that are irrelevant
FundamentusReits = FundamentusReits.drop(['Qtd de imóveis', 'Cap Rate', 'Vacância Média', 'Preço do m2', 'FFO Yield', 'Cotação', 'Aluguel por m2'], axis=1)

In [15]:
FundamentusReits.head()

,Papel,Segmento,Dividend Yield,P/VP,Valor de Mercado,Liquidez
0,ABCP11,Shoppings,0.0935,0.78,878798000,78309
1,AFHI11,Títulos e Val. Mob.,0.1559,0.99,284855000,844259
2,AGRX11,Híbrido,0.0325,1.00,48910500,15035
3,AIEC11,Lajes Corporativas,0.1240,0.70,345710000,564181
4,ALMI11,Lajes Corporativas,0.0000,0.41,102283000,164370


In [16]:
TickersData = pd.read_csv("TickersData20221126.csv", delimiter=";")
TickersData.columns

Index(['Ticker', 'Open', 'High', 'Low', 'Close', '52w High', '52 Low'], dtype='object')

In [17]:
TickersData = TickersData[['Ticker', 'Close', '52w High', '52 Low']]
TickersData.head()

,Ticker,Close,52w High,52 Low
0,AESB3,9.81,12.30,9.18
1,ONCO3,7.34,12.33,4.03
2,RAIL3,18.91,22.34,14.70
3,DMMO3,1.82,1.95,0.42
4,ELMD3,10.16,17.48,9.20


In [18]:
cur, conn = create_database()

Time to create the tables from the imported datasets!

In [19]:
FundamentusStocks_clean_create = ("""CREATE TABLE IF NOT EXISTS FundamentusStocks(
    Papel VARCHAR PRIMARY KEY,
    PL REAL,
    PVP REAL,
    DivYield REAL,
    EVEBITDA REAL
    )""")
cur.execute(FundamentusStocks_clean_create)
conn.commit()

In [20]:
FundamentusReits_create = ("""CREATE TABLE IF NOT EXISTS FundamentusReits(
    Papel VARCHAR PRIMARY KEY,
    Segmento VARCHAR,
    DividendYield REAL,
    PVP REAL,
    ValorDeMercado REAL,
    Liquidez REAL
    )""")
cur.execute(FundamentusReits_create)
conn.commit()

In [22]:
TickersData_create = ("""CREATE TABLE IF NOT EXISTS TickersData(
    Ticker VARCHAR PRIMARY KEY,
    Close REAL,
    YearHigh REAL,
    YearLow REAL
    )""")
cur.execute(TickersData_create)
conn.commit()

Initiating the insertion of data from the CSVs into the tables in Postgres

In [23]:
FundamentusStocks_table_insert = ("""INSERT INTO FundamentusStocks(
    Papel,
    PL,
    PVP,
    DivYield,
    EVEBITDA)
    VALUES (%s, %s, %s, %s, %s)
    """)
for i, row in FundamentusStocks_clean.iterrows():
    cur.execute(FundamentusStocks_table_insert, list(row))
conn.commit()

In [26]:
FundamentusReits_table_insert = ("""INSERT INTO FundamentusReits(
    Papel,
    Segmento,
    DividendYield,
    PVP,
    ValorDeMercado,
    Liquidez)
    VALUES (%s, %s, %s, %s, %s, %s)
    """)
for i, row in FundamentusReits.iterrows():
    cur.execute(FundamentusReits_table_insert, list(row))
conn.commit()

In [28]:
TickersData_table_insert = ("""INSERT INTO TickersData(
    Ticker,
    Close,
    YearHigh,
    YearLow)
    VALUES (%s, %s, %s, %s)
    """)
for i, row in TickersData.iterrows():
    cur.execute(TickersData_table_insert, list(row))
conn.commit()